In [21]:
import nltk

In [22]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [23]:
import numpy as np 
import pandas as pd
from nltk.corpus import stopwords # stopword : 불용어 제거 (the, a, !@#$$%$%$@ 이런 것들)
import string
import nltk

In [24]:
df = pd.read_csv("./data/spam.csv")
df

,Unnamed: 0,num,label,text,label_num
0,0,1,spam,your free lottery,1
1,1,2,spam,free lottery free you,1
2,2,3,ham,your free apple,0
3,3,4,ham,free to contact me,0
4,4,5,ham,I won award,0
5,5,6,spam,my lottery ticket,1


In [30]:
# 베이지안 확률 정의

def process_text(text) :
    
    # 구두점 제거
    nopunc = [char for char in text if char not in string.punctuation] # list conprehension
    nopunc = ''.join(nopunc)
    
    # text에서 무의미한 단어(접미사, 조사 등) 삭제 --> stopwords (불용어) 제거
    # 대소문자를 구분하기에 보통 소문자로 변환
    cleaned_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return cleaned_words


In [31]:
# 데이터 셋의 덱스트 데이터 토큰화(=분절) ex) 나는 / 너를 / 사랑해
df['text']

0         your free lottery
1    free lottery free you 
2          your free apple 
3       free to contact me 
4               I won award
5        my lottery ticket 
Name: text, dtype: object

In [32]:
# 데이터 셋의 덱스트 데이터 토큰화(=분절) ex) 나는 / 너를 / 사랑해
df['text'].apply(process_text)

0          [free, lottery]
1    [free, lottery, free]
2            [free, apple]
3          [free, contact]
4                  [award]
5        [lottery, ticket]
Name: text, dtype: object

In [34]:
# text를 token 수의 행렬로 변환

from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])
messages_bow

<6x6 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [36]:
# data를 train, test 로 분류

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['label_num'], test_size=0.2, random_state = 0)
# large X인 이유는 독립 변수라는 의미이다.
# test_size = 0.2 20%f로 하겠다 train data는 80%
# random_state = 0 : 한번 test, train를 나누면 그대로 사용하겠다.

In [37]:
# naive_bayes 다항식 나이브베이즈 모델 사용, 훈련

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train) # train 데이타를 fit 해야 함

MultinomialNB()

In [39]:
# 예측 값, 실제 값 출력

print(classifier.predict(X_train)) # 예측값
print(y_train.values)

[1 0 1 0]
[1 0 1 0]


In [44]:
# 학습 데이터셋에 모델 정확도

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

pred = classifier.predict(X_train) # 예측값 출력

# metrucs pkg 에 있는 정밀도(precision), 재현율(recall), f-1 socre 구하기
print(classification_report(y_train, pred))
# support는 데이터 수?

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



1. precision(정밀도) : 정확한 예측 / 예측을 기준으로 TP 진짜 양성인 것을 맞추는 것
2. recall(재현율) : 재현 실화냐 / 실제(actual)를 기준으로 TP 진짜 양성인 것을 맞추는 것
3. f1-score(정밀도와 재현율의 조화평균) :  사용이유 둘 다 중요한 지표니까 둘 다 고려함
4. 정밀도와 재현율은 상충관계(trade-off)
5. 산업군 마다 중요도 가 틀림
 - 의료 : 재현율 (암환자_True Positive 에게 함(Positive)라 진단해야 함)
 - 반도체 : 정밀도 (정확히 예측하는 제품을 생산해 정확하게 실제에 딱 맞춤)

In [45]:
# confusion matrix

confusion_matrix(y_train, pred)

array([[2, 0],
       [0, 2]], dtype=int64)

In [46]:
accuracy_score(y_train, pred)

1.0

In [47]:
# 테스트 데이터 셋 모델의 정확도 평가
classifier.predict(X_test)

array([1, 1], dtype=int64)

In [48]:
# 실제 관측값 출력

print(y_test.values)

[1 0]


In [52]:
# 테스트 셋에서 모델 평가

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

pred = classifier.predict(X_test) # X_test : test data(새로운 데이터로 간주)
print(confusion_matrix(y_test, pred)) # 실제 라벨, 예측 라벨

[[0 1]
 [0 1]]


In [53]:
print(accuracy_score(y_test, pred)) # 예측류 : 0.5 = 50%

0.5
